# **환경 설정**

In [ ]:
! pip install langchain_community tiktoken langchain-openai chromadb langchain langgraph tavily-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# **벡터 DB 구축**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.docment_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 크롤링할 블로그의 url 정의
urls = [
    "https://google.github.io/styleguide/pyguide.html",
    "https://google.github.io/styleguide/javaguide.html",
    "https://google.github.io/styleguide/jsguide.html",
]

# WebBaseLoader를 사용하여 주어진 URL 목록에서 문서 크롤링
docs = [WebBaseLoader(url).load() for url in urls]
# docs에서 sublist 이름으로 하나씩 추출
# -> sublist를 다시 item으로 추출
# 맨 왼쪽의 최종 반환값
docs_list = [item for sublist in docs for item in sublist]

# 지정한 크기만큼 텍스트를 분할하는 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

# 문서 분할
doc_splits = text_splitter.split_documents(docs_list)

# Chroma 벡터 저장소에 문서의 분할된 조각 저장
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEnbeddings(),
)

# 벡터 저장소에서 검색을 수행할 수 있는 검색기 생성
retriever = vectorstore.as_retriever()

# **문서 평가 노드**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# 문서와 질문의 연관성을 평가하기 위한 데이터 모델 정의
class GradeDocuments(BaseModel):
  binary_score: str = Field(
      description="문서와 질문의 연관성 여부. (예 or 아니오)"
  )

# 연관성 평가를 위한 LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperture=0)
# LLM의 출력이 반드시 GradeDocuments 형태가 되도록 강제한다
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# LLM이 사용자의 질문에 대해 문서의 연관성을 평가할 수 있도록 지시하는 프롬프트 정의
system = """당신은 사용자의 질문에 대해 검색된 문서의 관련성을 평가하는 전문가입니다.
  문서에 질문과 관련된 키워드나 의미가 담겨 있으면, 해당 문서를 '관련 있음'으로 평가하세요.
  문서가 질문과 관련이 있는지 여부를 '예' 또는 '아니오'로 표시해 주세요"""

# 시스템 메시지와 사용자의 질문 및 문서 내용을 포함한 템플릿 작성
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "검색된 문서: \n\n {document} \n\n 사용자 질문: {question}"),
    ]
)

# 프롬프트와 구조화된 LLM 평가기를 결합하여 retrieval_grader 객체 생성
retrieval_grader = grade_prompt | structured_llm_grader

In [ ]:
question = "파이썬 코드 작성 가이드"

# 연관 문서 검색
docs = retriever.invoke(question)
doc_txt = docs[1].page_content

# 검색된 문서의 연관성 평가
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

# **답변 생성**

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# LLM이 제공된 문맥을 바탕으로 답변할 수 있도록 지시하는 프롬프트 정의
system = """당신은 질문에 답변하는 업무를 돕는 도우미입니다.
제공된 문맥을 바탕으로 질문에 답변하세요. 만약 답을 모르면 모른다고 말하세요.
세 문장을 넘지 않도록 답변을 간결하게 작성하세요."""

# 시스템 메시지와 사용자의 질문 및 문서 내용을 포함한 템플릿 작성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "질문: {question} \n 문맥: {context} \n 답변: "),
    ]
)

# 검색된 문서들을 한 문자열로 병합
def format_docs(docs):
  return "\n\n.join(doc.page_content for doc in docs)"

# 프롬프트, LLM, 문자열 출력을 결합하여 RAG 체인 생성
rag_chain = prompt | llm | StrOutputParser()